### Import necessary liberaries

In [131]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Cursor
import random
from copy import deepcopy

### Let's state the steps to implement the Genetic Algorithm
1. We need to initialize the algorithm paramenters:
    - population_size
    - no_of_iterations / generations count (the Stopping Condition)
    - Elitism percentage(k) of population (best k paths/tours)
    - Crossover probability (generate new 2 tours from 2 older tours)
    - Mutation probability (swapping p of the cities in the same tour)
---------------------------------------------------------------------
2. Create class for Tour and City:
    - Tour class has:
        - list of cities
        - fitness score
        - distance/cost
        
    - City class has:
        - city name
        - x coordinate
        - y coordinate
----------------------------------------------------------------------
3. Initialze Population():
    - Generate tour from permutation(). Note: create deepcopy first
    - Calculate the fitness score (list) for the generated tour (1/tour cost)
        - Ex. c1 --->(2) c3 --->(1) c5 --->(6) c2
        - cost = 2 + 1 + 6 = 9  , ===>  so, fitness = 1/9
    - Add the tour to the population
----------------------------------------------------------------------
4. Fitness_evaluation():
    - calculate the cost of the current tour
    - store the cost
    - calculate the fitness (1/cost)
    - store the fitness
----------------------------------------------------------------------
5. Elitism():
    - caculate the number of elitism we're gonna choose based on the percentage
    - select top fittest tours
    - add elite tour to the new generation list (new pop)
----------------------------------------------------------------------
6. tournament_selection():
    - select random k tours
    - choose the fittest
----------------------------------------------------------------------
7. Crossover():  ---> use partial mapped crossover
    - crossover count (note: we will choose from the population except the elitism) so, count = (pop_size - no_of_elite tours)/2
    - iterate over the crossover count:
        - parent1 = tournament_selection()
        - parent2 = tournament_selection()
        - generate random number rn
            - if rn < crossover prob:
                - apply partial mapped crossover on parent1 and parent2
                - evaluate child1 and child2 fitness
                - insert the childerns to new generation list
            - else:
                - insert the parents to new generation list
----------------------------------------------------------------------
8. Mutation():  (swap mutation)
    - iterate over the population size:
        - parent1 = select random parent from new generation list
        - generate random double number from 0-->1
        - if the rn < mutation prob:
            - apply the swap mutation to parent1
            - evaluate its fitness
            - replace parent1 by mutated parent1 into the new gen list
  

In [21]:
# Choromosome
class Tour:
    def __init__(self, cities_list):
        self.fitness = 0
        self.cost = 0
        self.cities = cities_list
         
    def __repr__(self):
        return f"{self.cities}\nCost = {self.cost}"

In [22]:
# Gene
class City:
    def __init__(self, n, x, y):
        self.name = n
        self.x_coor = x
        self.y_coor = y
        
    def __repr__(self):
        return f"{self.name}"

### Read the data

In [63]:
def read_data(data):
    data_list = []
    m = data.shape[0]
    
    for i in range(m):
        data_list.append(City(data['City'][i], data['x'][i], data['y'][i]))
        
    return data_list

In [24]:
def get_distance(city1, city2):
    x_diff = city1.x_coor - city2.x_coor
    y_diff = city1.y_coor - city2.y_coor
    distance = np.sqrt((x_diff ** 2) + (y_diff ** 2))
#     print(f"***** Distance Between {city1.name} & {city2.name} *****")
#     print("Distance:", distance)
    return distance

In [25]:
def distance_matrix(tour):
    m = len(tour)
    matrix = np.zeros ((m, m))
    for i in range(m):
        for j in range(m):
            matrix[i][j] = get_distance(tour[i], tour[j])
            
    return matrix

In [26]:
def get_tour_cost(tour, dist_matrix):
#     print("***** Get Tour Cost *****")
    cost = 0
    for i in range(len(tour.cities)-1):
        city1 = tour.cities[i]
        city2 = tour.cities[i+1]
        cost += dist_matrix[(city1.name)-1, (city2.name)-1] # indicies
    # cost of returning back to the starting point
    cost += get_distance(tour.cities[-1], tour.cities[0])

    return cost

In [27]:
def get_tour_fitness(tour, dist_matrix):
#     print("***** Get Tour Fitness *****")
    fitness = 1/get_tour_cost(tour, dist_matrix)
    return fitness

In [28]:
def initialize_population(data_lst, pop_size, dist_matrix):
#     print("***** Initial Population *****")
    population = []
    for i in range(pop_size):
        tour = Tour(list(np.random.permutation(data_lst)))
        tour.cost = get_tour_cost(tour, dist_matrix)
        tour.fitness = get_tour_fitness(tour, dist_matrix)
        population.append(tour)
        
    return population

In [29]:
def elitism(population, elite_num):
#     print("***** Elitism Selection *****")
    sorted_pop = sorted(population, key=lambda tour: tour.fitness, reverse=True)
    return sorted_pop[:elite_num]

In [30]:
def tournament_selection(pop):
    candidates = random.choices(pop, k=5)
#     print("***** Tournament Selection *****")
    return max(candidates, key = lambda x: x.fitness)

In [31]:
def PMC_crossover(parent1, parent2, distance_matrix):
    crossover_point = random.choices(range(len(parent1.cities)), k=2)
    r1 = min(crossover_point)
    r2 = max(crossover_point)
    
    child1 = deepcopy(parent1)
    child2 = deepcopy(parent2)
    
    for i in range(r1, r2):
        temp1 = parent2.cities[i]
        temp2 = parent1.cities[i]
        
        indx1 = [c.name for c in child1.cities].index(temp1.name)
        indx2 = [c.name for c in child2.cities].index(temp2.name)
        
        child1.cities[indx1] = child1.cities[i]   
        child2.cities[indx2] = child2.cities[i]
        
        child1.cities[i] = temp1
        child2.cities[i] = temp2
       
    child1.cost = get_tour_cost(child1, distance_matrix)
    child2.cost = get_tour_cost(child2, distance_matrix)

    child1.fitness = 1/child1.cost
    child2.fitness = 1/child2.cost
    
    return child1, child2

In [32]:
def crossover(pop_size, crossover_prob, oldpop, distance_matrix):
#     print("***** Crossover *****")
    new_pop = []
    while True:
        parent1 = tournament_selection(oldpop)
        parent2 = tournament_selection(oldpop)
        rand_num = random.uniform(0,1)

        # apply partial crossover
        if rand_num < crossover_prob:
            child1, child2 = PMC_crossover(parent1, parent2, distance_matrix)   

            new_pop.append(child1)
            new_pop.append(child2)

        else:
            new_pop.append(parent1)
            new_pop.append(parent2)
            
        if len(new_pop) == pop_size:
            return new_pop

In [33]:
def tour_mutation(tour):
#     print("***** Tour Mutation *****")
    tour_length = len(tour.cities)
    swap_points = np.random.choice(np.arange(0, tour_length), size= 2, replace=False)
    c1, c2 = min(swap_points), max(swap_points)
    # swap cities then return the route again
    tour.cities[c1], tour.cities[c2] = tour.cities[c2], tour.cities[c1]
    return tour

In [43]:
def mutation(pop_size, pop, elite_num, mutation_prob, dist_matrix):
#     print("***** Mutation *****")
    mutation_size = mutation_prob * pop_size
    rand_tours = np.random.choice(np.arange(elite_num, pop_size), size = round(mutation_size), replace=False)
    rand_num = random.uniform(0,1)
#     if rand_num < mutation_prob:
    for tour in rand_tours:
        pop[tour] = tour_mutation(pop[tour])
        pop[tour].cost = get_tour_cost(pop[tour], dist_matrix)
        pop[tour].fitness = 1 / pop[tour].cost

    return pop

In [35]:
def Gene_Algo(pop_size, oldpop, num_elite, mutation_prob, crossover_prob, distance_matrix):
    newpop = []

    elites = elitism(oldpop, num_elite)
    newpop.extend(elites)

    newpop = crossover(pop_size, crossover_prob, oldpop, distance_matrix)
    newpop = mutation(pop_size, newpop, num_elite, mutation_prob, distance_matrix)

    return newpop

In [105]:
def run_algorithm(data):
    # initialize parameters
    mutation_prob = 0.3
    crossover_prob = 0.7
    population_size = 400
    num_elite = 100
    num_generations = 100
    
    lst = read_data(data)
    dist_matrix = distance_matrix(lst)
    population = initialize_population(lst, population_size, dist_matrix)
    
    for i in range (num_generations):   
        population = Gene_Algo(population_size, population, num_elite, mutation_prob, crossover_prob, dist_matrix)
        population = elitism(population, num_elite)

    result = elitism(population, num_elite)[0]
    result.cities.append(result.cities[0])
    print("Final Cost:", result)
    return result

In [106]:
df = pd.read_csv('15-Points.csv')

In [107]:
ans = run_algorithm(df)

Final Cost: [14, 12, 3, 7, 5, 9, 15, 2, 13, 1, 11, 4, 6, 8, 10, 14]
Cost = 284.3810904080331


### Plot the Shortest Path

In [108]:
X = [city.x_coor  for city in ans.cities]
Y = [city.y_coor  for city in ans.cities]

In [109]:
plt.plot(X, Y, '-o')
plt.title(f'Genetic Algorithm for TSP Path with cost = {ans.cost}')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.grid()
plt.show()

In [110]:
%matplotlib qt

In [176]:
def ginput_fun():
    points = []
    
    # display a plot and wait for the user to click on num_points points
    fig, ax = plt.subplots()
    plt.title(f"Enter points")
    plt.xlim(0, 10)
    plt.ylim(0, 10)
    plt.draw()
    
    cursor = Cursor(ax, horizOn=True, vertOn=True, useblit=True, color='black', linewidth=1)
    
    # take input fro
    while True:
        # get the first tuple (first point)
        point = plt.ginput(1)
        if point:
            point=point[0]
        else:
            break
#         print(point)
        points.append(point)
        plt.plot(point[0], point[1], 'go')
        plt.draw()  
        
    # put the points into a dataframe
    point_dicts = [{'x': point[0], 'y': point[1]} for point in points]
    n=len(points)
    city = [i+1 for i in range(n)]
    data = pd.DataFrame(point_dicts)

    # add column called city at index 2
    data.insert(2, 'City', city)
    
    # save data to csv file
    data.to_csv('my_data.csv', index = False)
    
    # get the nearest neighbor
    best_sol = run_algorithm(data)
    
    X = [city.x_coor  for city in best_sol.cities]
    Y = [city.y_coor  for city in best_sol.cities]
    
    # list of tuples [(x, y)]
    my_path = [(X[i], Y[i]) for i in range(len(X))]

    
    for i in range(len(my_path)+1):
        try:
            plt.plot([my_path[i][0], my_path[i+1][0]], [my_path[i][1], my_path[i+1][1]], '-o', color='green')
            plt.pause(0.3)
        except:
            continue

    plt.title(f'Genetic Algorithm (TSP) Path, Cost = {best_sol.cost}')
    i = 1
    for x, y in points:
        plt.text(x, y+0.3 ,f'C{i}')
        i+=1
        
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.show()   

In [198]:
ginput_fun()

Final Cost: [11, 14, 16, 15, 17, 9, 8, 7, 6, 5, 4, 3, 2, 10, 1, 12, 13, 11]
Cost = 43.309047574691974
